# Project

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
FILENAME= "calendar.csv"
DATADIR = "../pfda_project/"

calendar_df = pd.read_csv(DATADIR + FILENAME, header=0)

In [4]:
FILENAME= "inventory.csv"
DATADIR = "../pfda_project/"

inventory_df = pd.read_csv(DATADIR + FILENAME, header=0)

In [5]:
FILENAME= "sales_test.csv"
DATADIR = "../pfda_project/"

sales_test_df = pd.read_csv(DATADIR + FILENAME, header=0)

In [6]:
FILENAME= "sales_train.csv"
DATADIR = "../pfda_project/"

sales_train_df = pd.read_csv(DATADIR + FILENAME, header=0)

In [7]:
FILENAME= "solution.csv"
DATADIR = "../pfda_project/"

solution_df = pd.read_csv(DATADIR + FILENAME, header=0)

In [8]:
FILENAME= "test_weights.csv"
DATADIR = "../pfda_project/"

test_weights_df = pd.read_csv(DATADIR + FILENAME, header=0)